## ARP spoofing detector 
### with Random Forest

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import sklearn.metrics as mt

In [2]:
#탐지할 데이터 셋 설정
raw_data = pd.read_csv('mitm-arpspoofing-1-dec.csv',low_memory=False)

In [3]:
#데이터 전처리
ip_src = []
ip_dst = []
eth_src = []
eth_dst = []
arp_src = []
arp_dst = []
ip_proto = []

for i in range(len(raw_data['ip.src'])):
    if raw_data['ip.src'][i] == '192.168.0.16' and raw_data['ip.dst'][i] == '192.168.0.13':
        ip_src.append(1)
        ip_dst.append(1)
    elif raw_data['ip.src'][i] == '192.168.0.13' and raw_data['ip.dst'][i] == '192.168.0.16':
        ip_src.append(1)
        ip_dst.append(1)
    else:
        ip_src.append(0)
        ip_dst.append(0)
for each in raw_data['eth.src']:
    if each == 'f0:18:98:5e:ff:9f':
        eth_src.append(1)
    else:
        eth_src.append(0)
for each in raw_data['eth.dst']:
    if each == 'f0:18:98:5e:ff:9f':
        eth_dst.append(1)
    else:
        eth_dst.append(0)
for each in raw_data['arp.src.hw_mac']:
    if each == 'f0:18:98:5e:ff:9f':
        arp_src.append(1)
    else:
        arp_src.append(0)
for each in raw_data['arp.dst.hw_mac']:
    if each == 'bc:1c:81:4b:ae:ba':
        arp_dst.append(1)
    elif each == '48:4b:aa:2c:d8:f9':
        arp_dst.append(2)
    else:
        arp_dst.append(0)
for each in raw_data['ip.proto']:
    if each == '1':
        ip_proto.append(1)
    elif each == '6':
        ip_proto.append(6)
    else:
        ip_proto.append(0)

raw_data['ip.src'] = ip_src
raw_data['ip.dst'] = ip_dst
raw_data['eth.src'] = eth_src
raw_data['eth.dst'] = eth_dst
raw_data['arp.src.hw_mac'] = arp_src
raw_data['arp.dst.hw_mac'] = arp_dst
raw_data['ip.proto'] = ip_proto

train_pre = raw_data[['ip.src','ip.dst','eth.src','eth.dst','arp.src.hw_mac','arp.dst.hw_mac']]
train_pre

,ip.src,ip.dst,eth.src,eth.dst,arp.src.hw_mac,arp.dst.hw_mac
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0
...,...,...,...,...,...,...
65763,1,1,0,0,0,0
65764,1,1,0,0,0,0
65765,1,1,0,0,0,0
65766,1,1,0,0,0,0


In [8]:
#trainData, testData, trainLabel, testLabel 훈련셋, 테스트셋 분리
x_train, x_test, y_train, y_test = train_test_split(train_pre,raw_data[['label']],random_state=0)

rf_clf = RandomForestClassifier(criterion='entropy', n_estimators=5, n_jobs=2, random_state=0)
rf_clf = rf_clf.fit(x_train, y_train.values.ravel())
y_pred = rf_clf.predict(x_test)

In [10]:
# 교차검증
from sklearn.model_selection import cross_val_score, cross_validate

# 각 폴드의 스코어 
scores = cross_val_score(rf_clf, x_train, y_train.values.ravel(), cv = 5)

pd.DataFrame(cross_validate(rf_clf, x_train, y_train.values.ravel(), cv =5))
print('cross validation avg: ', scores.mean())

cross validation avg:  0.9999594546651434


In [11]:
print('Train_Accuracy:{}'.format(rf_clf.score(x_train,y_train)))

accuracy = mt.accuracy_score(y_test, y_pred)
recall = mt.recall_score(y_test, y_pred)
precision = mt.precision_score(y_test, y_pred)
f1_score = mt.f1_score(y_test, y_pred)

print('Accuracy: ', format(accuracy))
print('Recall: ', format(recall))
print('Precision: ', format(precision))
print('F1_score: ', format(f1_score))

Train_Accuracy:0.9999797267161334
Accuracy:  1.0
Recall:  1.0
Precision:  1.0
F1_score:  1.0


In [12]:
#탐지 결과 출력
print("<Result>")
for each in y_pred :
    print(each)

<Result>
1
0
1
0
0
0
1
1
1
0
1
0
1
0
1
0
1
1
0
0
0
1
1
0
0
1
1
0
1
0
1
1
0
1
1
0
1
0
1
0
0
1
1
0
1
0
0
1
1
1
0
1
1
1
0
0
1
0
0
1
0
1
0
1
0
0
0
1
1
0
1
0
1
0
0
0
0
0
1
0
0
1
0
1
0
1
0
1
1
0
1
0
1
1
0
0
0
1
1
0
0
1
1
0
0
0
0
1
0
1
1
1
0
0
1
0
1
0
1
1
0
1
0
0
1
1
0
1
0
0
1
0
0
1
0
1
0
0
0
1
0
1
1
0
1
1
1
0
0
0
1
0
1
1
0
1
0
0
1
1
1
1
1
0
0
0
0
0
0
1
0
0
1
1
0
1
0
0
0
1
0
0
0
0
1
1
1
0
0
0
0
0
0
0
1
0
1
1
0
0
0
1
1
1
1
1
0
0
1
0
0
1
0
0
0
1
1
0
1
1
1
1
0
0
1
0
0
0
1
0
0
0
0
1
0
0
1
1
1
0
0
0
0
0
0
1
1
1
0
0
0
0
0
1
1
1
0
0
0
1
1
0
1
1
0
1
1
1
0
0
1
0
0
1
1
0
0
1
1
0
1
0
0
0
0
0
0
0
0
1
1
0
0
1
0
1
0
0
1
0
1
1
1
1
0
1
0
1
1
0
0
0
0
0
0
0
1
1
1
1
0
1
0
0
0
1
0
0
1
1
1
1
0
0
1
1
0
0
0
1
1
0
0
0
0
1
1
1
0
1
1
0
0
1
1
1
0
0
1
0
0
1
1
1
0
0
0
1
1
0
0
0
0
1
1
1
0
0
0
1
1
1
0
1
1
1
0
0
1
0
0
0
1
0
0
1
0
0
0
0
0
0
1
0
1
1
0
1
1
0
0
0
0
0
0
1
0
0
0
1
1
0
1
0
1
1
0
1
1
1
0
0
1
0
0
1
0
0
0
1
0
0
0
1
0
1
0
1
1
1
1
0
1
0
1
0
0
1
0
1
0
1
1
0
1
0
0
0
1
1
0
1
1
1
1
1
0
1
0
1
0
0
0
1
0
0
1
1
0
1
1
0
1
1
0
0

0
1
0
0
1
1
1
1
1
1
1
1
1
1
0
0
1
0
1
0
1
0
0
0
0
1
0
0
1
1
1
0
0
0
1
0
1
0
0
1
0
0
0
1
0
0
1
0
0
0
1
0
1
0
1
0
1
1
0
1
0
0
1
0
0
1
0
1
1
0
1
1
0
1
1
0
1
1
0
1
0
1
0
0
0
0
1
1
0
1
0
1
0
1
0
1
0
0
1
0
1
1
0
0
1
1
1
1
0
1
0
0
0
1
1
1
1
0
0
0
0
1
1
1
1
1
1
0
1
0
1
0
1
1
1
0
0
0
1
1
1
1
0
1
1
1
1
0
0
0
0
1
0
1
1
0
0
1
1
1
1
0
1
1
1
0
0
0
1
1
1
0
1
0
1
1
1
1
0
1
0
0
0
1
1
1
1
0
1
1
1
1
1
0
1
1
1
0
1
1
1
1
1
0
0
0
0
1
0
1
1
1
0
1
1
0
1
0
0
1
0
0
1
0
0
0
1
0
1
1
1
1
1
1
1
0
1
0
1
1
1
1
0
1
1
0
1
1
1
1
0
1
0
1
1
1
1
0
0
0
1
1
1
1
0
1
1
1
0
0
0
0
1
0
1
1
1
1
0
0
1
1
0
1
1
0
1
0
1
0
1
1
0
0
0
0
1
1
0
1
0
0
0
0
1
1
0
1
1
1
0
0
1
0
0
1
0
1
0
1
1
1
1
1
1
0
1
0
0
0
1
1
1
0
0
0
1
1
1
0
1
0
1
0
0
0
1
0
0
0
1
0
0
1
0
1
0
0
1
1
0
1
1
0
0
0
0
0
1
0
0
1
0
0
1
0
1
0
0
0
0
0
0
0
1
0
0
0
0
1
0
1
0
0
0
1
0
0
0
0
1
1
1
1
0
1
0
0
1
0
1
1
0
0
1
1
0
1
0
1
1
1
1
0
0
0
0
1
1
0
1
1
1
0
1
1
0
1
0
0
1
0
1
0
0
0
1
0
1
0
1
1
1
0
1
0
1
0
1
1
0
1
1
0
1
0
1
0
0
1
1
1
0
1
0
1
1
1
0
0
0
0
0
0
1
0
0
1
0
0
1
1
1
0
0
0
1
1
1
0


1
1
1
0
0
0
1
1
0
0
1
0
1
1
1
1
1
1
1
1
0
1
0
0
0
0
0
1
0
1
0
1
0
1
0
0
1
0
1
1
1
0
1
0
0
0
1
1
0
1
1
1
1
0
0
0
0
1
0
1
0
0
1
1
1
0
1
1
0
0
1
1
1
1
1
1
0
1
0
1
0
0
1
1
1
0
0
1
1
0
1
1
0
0
1
1
1
1
0
0
1
1
1
1
1
0
1
1
1
1
1
1
0
0
1
1
0
1
1
0
1
1
1
0
0
1
0
0
1
0
0
0
0
0
1
1
1
1
1
0
0
0
1
0
0
1
0
0
0
0
1
1
0
1
1
0
1
1
1
0
0
0
0
1
1
1
1
0
0
1
1
0
1
1
1
1
1
1
1
0
0
0
1
0
1
0
1
0
0
1
0
1
0
1
0
0
1
1
0
1
0
0
0
0
0
0
1
0
0
1
0
1
1
1
0
1
1
1
1
1
0
0
0
1
0
1
0
0
1
1
1
0
0
1
0
0
1
1
1
0
1
0
1
0
1
1
1
0
1
0
1
1
0
1
1
1
1
1
0
0
0
1
0
1
1
1
0
1
1
1
0
1
0
1
0
1
0
0
1
0
0
0
1
1
1
0
1
0
0
0
1
1
1
0
0
1
1
0
0
0
1
1
0
1
1
1
0
0
1
0
1
0
0
0
1
0
1
1
1
0
1
1
0
1
0
0
0
0
1
1
0
0
0
1
1
0
0
0
1
1
1
0
1
0
1
1
1
1
0
0
0
1
1
1
1
1
1
0
0
1
0
1
0
0
1
0
0
1
0
0
1
0
0
0
1
0
0
0
1
1
1
1
0
0
0
0
0
1
1
1
1
0
1
1
1
1
0
0
0
1
1
0
1
1
1
0
1
0
1
1
0
1
1
0
1
0
1
0
0
0
0
0
0
1
1
1
0
1
0
0
0
0
1
1
0
1
0
1
0
1
1
1
1
1
1
1
0
0
1
1
1
1
0
0
0
1
1
0
1
1
0
1
1
1
1
0
1
1
1
0
0
1
1
0
1
0
1
1
0
1
0
0
0
1
0
1
0
0
1
1
1
1
0
1
0
1
1
0
1
1


0
1
1
0
1
0
1
0
0
0
0
0
0
0
1
0
0
0
1
1
1
0
0
1
1
1
0
1
1
1
1
1
1
0
1
0
0
1
0
1
0
0
1
0
0
1
0
1
1
0
0
0
1
1
1
1
0
1
1
0
0
0
0
1
0
0
0
0
0
0
1
1
1
0
0
0
1
0
1
1
0
0
0
0
1
0
0
0
0
0
0
0
1
1
1
1
1
0
1
1
0
0
0
1
1
0
0
0
0
1
1
0
1
0
0
1
1
1
1
1
1
1
1
0
0
0
0
0
1
1
1
1
0
1
0
1
0
0
1
1
1
1
1
1
0
1
0
1
0
1
0
0
1
1
1
1
0
0
1
1
1
0
0
1
1
0
1
1
1
1
1
0
1
0
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
1
1
1
0
1
0
1
1
1
1
1
0
0
0
0
1
1
1
0
0
1
1
1
1
0
0
0
0
1
0
1
1
0
0
1
0
0
1
1
1
1
0
1
0
1
0
1
0
1
0
0
1
0
0
0
1
0
0
1
0
1
0
1
1
0
0
1
1
1
1
0
0
1
1
0
1
1
0
1
1
1
1
0
0
1
0
0
1
1
0
1
1
0
0
0
0
1
1
1
0
0
0
0
1
0
1
1
1
1
1
0
1
0
1
1
0
1
1
0
0
1
1
0
0
1
0
1
1
0
0
0
1
0
1
1
1
1
1
0
0
0
0
1
0
0
1
0
1
1
0
1
0
1
1
0
0
1
0
0
1
0
1
1
1
1
0
1
0
0
0
1
0
1
0
0
0
1
1
0
1
1
0
0
1
0
1
1
0
0
0
1
0
0
0
0
0
1
1
0
0
1
1
0
0
0
0
1
1
1
1
0
0
1
0
1
0
0
1
0
0
1
0
0
0
1
0
0
0
0
1
0
0
0
0
1
1
0
0
0
0
0
1
0
0
0
0
1
1
1
0
0
0
1
1
0
0
0
1
0
0
0
0
1
1
0
0
1
1
0
0
0
0
1
0
0
1
1
0
1
0
0
1
0
0
0
0
1
0
1
1
0
0
1
1
1
0
0
0
0
1
0
1
0
